In [1]:
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.python.keras.layers import Dense, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.layers import Dense, Flatten, BatchNormalization, Dropout, ActivityRegularization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.regularizers import l1

from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt

from tensorflow.keras import regularizers

from tensorflow.keras.applications import VGG16



In [2]:

# Download the train, validation and test dataset from google colab which were created with the dataset preprocess files

dataset = [
    '1-IpL9lR5gEFMKGLfDaIIPlrzf2zfnGaN', #https://drive.google.com/file/d/1-IpL9lR5gEFMKGLfDaIIPlrzf2zfnGaN/view?usp=sharing
    '1-IilI3zbFwBWIRmi4VvKkI6GkHJ6hXwQ', #https://drive.google.com/file/d/1-IilI3zbFwBWIRmi4VvKkI6GkHJ6hXwQ/view?usp=sharing
    '1kKCM-H7cgHTw5p0wQsyNH2Ali-30Bbz-', #https://drive.google.com/file/d/1kKCM-H7cgHTw5p0wQsyNH2Ali-30Bbz-/view?usp=sharing

]


# Download the dataset audio files
for i, id in enumerate(dataset):
  destination = f'{i}.wav'

  # Check if the file already exists
  if not os.path.exists(destination):
 
    !wget --content-disposition 'https://drive.google.com/uc?export=download&id={id}&confirm=t'


In [2]:

# Function to parse TFRecord entries, declaring the metadatas of the TFRecord files
def _parse_function(proto):
    keys_to_features = {
        'data': tf.io.FixedLenFeature([128 * 313 * 3], tf.float32),
        'labels': tf.io.FixedLenFeature([], tf.int64),
    }
    parsed_features = tf.io.parse_single_example(proto, keys_to_features)
    parsed_features['data'] = tf.reshape(parsed_features['data'], (128, 313, 3))
    return parsed_features['data'], parsed_features['labels']





In [3]:
# Load and parse TFRecord datasets for testing, validation,and training
# These dataset are pipelined so they are not loaded to the memory.

test_dataset = tf.data.TFRecordDataset('test_dataset.tfrecord').map(_parse_function)
val_dataset = tf.data.TFRecordDataset('val_dataset.tfrecord').map(_parse_function)
train_dataset = tf.data.TFRecordDataset('train_dataset.tfrecord').map(_parse_function)

In [4]:
model = Sequential()

pretrained_model= tf.keras.applications.VGG16(
    include_top=False,
    input_shape=(128,313,3),
    pooling='avg',classes=264,
    weights='imagenet')

for layer in pretrained_model.layers:
        layer.trainable=False



model.add(pretrained_model)

model.add(Flatten())

model.add(BatchNormalization())


model.add(Dense(600, activation='relu',kernel_regularizer=keras.regularizers.l2(0.00001)))

model.add(Dropout(0.5))

model.add(Dense(600, activation='relu',kernel_regularizer=keras.regularizers.l2(0.00001)))

model.add(Dropout(0.5))


model.add(Dense(264, activation='softmax'))


In [5]:
# Compile the model

optimizer = Adam(learning_rate=0.001)

model.compile(
    optimizer=optimizer,
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Display model summary
pretrained_model.summary()


Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 128, 313, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 128, 313, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 128, 313, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 156, 64)       0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 64, 156, 128)      73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 64, 156, 128)      147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 32, 78, 128)       0     

In [6]:
# Prefetch datasets for better performance
train_dataset = train_dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
val_dataset = val_dataset.prefetch(buffer_size=tf.data.AUTOTUNE)

# Shuffle training dataset
train_dataset = train_dataset.shuffle(5)



In [7]:

# Define early stopping and checkpoint callbacks
patience = 5
early_stopping=EarlyStopping(patience=patience, verbose=2)
checkpointer=ModelCheckpoint(filepath='weights.hdf5', save_best_only=True, verbose=2)

# Training the dataset:


In [ ]:
# Train the model
num_epochs = 30
batch_size = 128

history = model.fit(
    train_dataset.batch(batch_size),
    epochs=num_epochs,
    validation_data=val_dataset.batch(batch_size),
    callbacks=[checkpointer, early_stopping]
)

Epoch 1/30
202/202 [==============================] - 177s 774ms/step - loss: 5.0852 - accuracy: 0.0614 - val_loss: 5.3161 - val_accuracy: 0.0398

Epoch 00001: val_loss improved from inf to 5.31608, saving model to weights.hdf5
Epoch 2/30
111/202 [===============>..............] - ETA: 48s - loss: 4.6145 - accuracy: 0.1101

In [ ]:
# Plot training and validation loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.grid()
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.legend(['train', 'validation'])
plt.show()

# Evaluation:

In [ ]:
# Download weights if needed

!wget --content-disposition 'https://drive.google.com/uc?export=download&id=16nsgJFzI7bIwovAvIVJNNNOpXL6B3abM&confirm=t'

In [19]:
# Prefetch datasets for better performance
train_dataset = train_dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size=tf.data.AUTOTUNE)

# Shuffle training dataset
train_dataset = train_dataset.shuffle(5)


model.load_weights('weights.hdf5')
batch_size = 128

evaluation_result = model.evaluate(test_dataset.batch(batch_size))

loss_value = evaluation_result[0]
accuracy_value = evaluation_result[1]

ValueError: Cannot assign to variable dense_11/kernel:0 due to variable shape (1024, 512) and value shape (1024, 1024) are incompatible

In [ ]:
print(f'the test accuracy is {accuracy_value}')